In [4]:
from transformers import BertTokenizer, AutoConfig, AutoModelForSequenceClassification 

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [5]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd

def data_loading(url):
    with open(url, 'r', encoding='utf-8') as f:
        # data = json.loads(f.read())
        df = pd.read_json(f)
        data = df.copy()
        # data = df.loc[:, ['sentiment', 'body']]
        data = data.loc[df['sentiment'].notnull()]
        data['sentiment'] = pd.Categorical(data['sentiment'])
        data['label'] = data['sentiment'].cat.codes
        data = data.rename(columns={'sentiment': 'labels', 'body': 'sentense'})

        return data

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data = data_loading(url)

dataset = Dataset.from_pandas(data.loc[:, ['label', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')

In [6]:
def encode(example):
    result = tokenizer(example['sentense'], padding='max_length', truncation=True)
    
    return result

encoded_dataset = dataset.map(encode, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)
# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# type(encoded_dataset[0]['labels'])

100%|██████████| 1/1 [00:00<00:00,  1.21ba/s]


In [7]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric

import numpy as np

training_args = TrainingArguments(
    output_dir="bert-classifier", 
    evaluation_strategy="epoch",
    save_steps=50,
    epoch=1
)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
)

trainer.train()


TypeError: __init__() got an unexpected keyword argument 'epoch'